In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import re

In [22]:
# Wir holen alle EM-Matches von 2000 - 2021

In [23]:
standing_url = "https://fbref.com/en/comps/676/history/UEFA-Euro-Seasons"

data = requests.get(standing_url)

soup = BeautifulSoup(data.text)

standing_table = soup.select("table.stats_table")[0]

standing_table

<table class="sortable stats_table" data-cols-to-freeze=",1" id="seasons">
<caption>UEFA European Football Championship Seasons Table</caption>
<colgroup><col/><col/><col/><col/><col/><col/><col/><col/></colgroup>
<thead>
<tr>
<th aria-label="Year" class="poptip center" data-stat="year" scope="col">Year</th>
<th aria-label="Competition Name" class="poptip center" data-stat="comp_name" data-tip="Name the competition was known as for that year" scope="col">Competition Name</th>
<th aria-label="Host Country" class="poptip center" data-stat="host_country" data-tip="Name of country hosting competition" scope="col">Host Country</th>
<th aria-label="# Squads" class="poptip center" data-stat="num_teams" data-tip="No. of squads participating in competition." scope="col"># Squads</th>
<th aria-label="Champion" class="poptip center" data-stat="champ" data-tip="League champion for that season" scope="col">Champion</th>
<th aria-label="Runner-Up" class="poptip center" data-stat="runner_up" data-tip

In [24]:
links = standing_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if "/comps/" in l]
links

['/en/comps/676/UEFA-Euro-Stats',
 '/en/comps/676/UEFA-Euro-Stats',
 '/en/comps/676/2021/2021-UEFA-Euro-Stats',
 '/en/comps/676/2021/2021-UEFA-Euro-Stats',
 '/en/comps/676/2016/2016-UEFA-Euro-Stats',
 '/en/comps/676/2016/2016-UEFA-Euro-Stats',
 '/en/comps/676/2012/2012-UEFA-Euro-Stats',
 '/en/comps/676/2012/2012-UEFA-Euro-Stats',
 '/en/comps/676/2008/2008-UEFA-Euro-Stats',
 '/en/comps/676/2008/2008-UEFA-Euro-Stats',
 '/en/comps/676/2004/2004-UEFA-Euro-Stats',
 '/en/comps/676/2004/2004-UEFA-Euro-Stats',
 '/en/comps/676/2000/2000-UEFA-Euro-Stats',
 '/en/comps/676/2000/2000-UEFA-Euro-Stats']

In [25]:
# deletes duplicate
new_links = []
for l in links:
    if l not in new_links:
        new_links.append(l)
new_links

['/en/comps/676/UEFA-Euro-Stats',
 '/en/comps/676/2021/2021-UEFA-Euro-Stats',
 '/en/comps/676/2016/2016-UEFA-Euro-Stats',
 '/en/comps/676/2012/2012-UEFA-Euro-Stats',
 '/en/comps/676/2008/2008-UEFA-Euro-Stats',
 '/en/comps/676/2004/2004-UEFA-Euro-Stats',
 '/en/comps/676/2000/2000-UEFA-Euro-Stats']

In [26]:
# delete the 2024 EM Stats, because we want to predict 2024
new_links.remove("/en/comps/676/UEFA-Euro-Stats")
new_links

['/en/comps/676/2021/2021-UEFA-Euro-Stats',
 '/en/comps/676/2016/2016-UEFA-Euro-Stats',
 '/en/comps/676/2012/2012-UEFA-Euro-Stats',
 '/en/comps/676/2008/2008-UEFA-Euro-Stats',
 '/en/comps/676/2004/2004-UEFA-Euro-Stats',
 '/en/comps/676/2000/2000-UEFA-Euro-Stats']

In [27]:
em_urls = [f"https://fbref.com{l}" for l in new_links]
em_urls

['https://fbref.com/en/comps/676/2021/2021-UEFA-Euro-Stats',
 'https://fbref.com/en/comps/676/2016/2016-UEFA-Euro-Stats',
 'https://fbref.com/en/comps/676/2012/2012-UEFA-Euro-Stats',
 'https://fbref.com/en/comps/676/2008/2008-UEFA-Euro-Stats',
 'https://fbref.com/en/comps/676/2004/2004-UEFA-Euro-Stats',
 'https://fbref.com/en/comps/676/2000/2000-UEFA-Euro-Stats']

In [28]:
# later with for loop
all_matches = []
for em_url in em_urls:

    print(em_url)

    # Von URL soll das Jahr rausgenommen werden
    string_e = em_url.split("/")
    year = string_e[6] if len(string_e) > 6 else "Unknown"
    
    # Validate the year
    if year.isdigit() and len(year) == 4 and int(year) >= 2000:
        valid_year = year
    else:
        valid_year = "Unknown"

    data = requests.get(em_url)

    matches = pd.read_html(StringIO(str(data.text)), match="League Table Table")[0]

    matches = matches.dropna(how='all')
    
    matches['Squad'] = matches['Squad'].apply(lambda x: x[3:].strip() if isinstance(x, str) and len(x) > 3 else x)
    
    # Add the year column next to Squad
    matches.insert(matches.columns.get_loc('Squad') + 1, 'Year', year)
    
    # Index zurücksetzen
    matches = matches.reset_index(drop=True)
    
    matches = matches.sort_values(['Pts', 'GD'], ascending=[False, False])

    # Platzierung zuweisen
    matches['Rk'] = range(1, len(matches) + 1)

    # Spalten neu anordnen, sodass 'Rk' an erster Stelle steht
    columns = matches.columns.tolist()
    columns = ['Rk'] + [col for col in columns if col != 'Rk']
    matches = matches[columns]

    matches
    all_matches.append(matches)

https://fbref.com/en/comps/676/2021/2021-UEFA-Euro-Stats
https://fbref.com/en/comps/676/2016/2016-UEFA-Euro-Stats
https://fbref.com/en/comps/676/2012/2012-UEFA-Euro-Stats
https://fbref.com/en/comps/676/2008/2008-UEFA-Euro-Stats
https://fbref.com/en/comps/676/2004/2004-UEFA-Euro-Stats
https://fbref.com/en/comps/676/2000/2000-UEFA-Euro-Stats


In [29]:
len(all_matches)

6

In [30]:
all_matches

[    Rk         Squad  Year   MP    W    D    L    GF    GA   GD   Pts    xG  \
 0    1         Italy  2021  7.0  5.0  2.0  0.0  13.0   4.0  9.0  17.0  13.1   
 1    2       England  2021  7.0  5.0  2.0  0.0  11.0   2.0  9.0  17.0  11.0   
 4    3       Belgium  2021  5.0  4.0  0.0  1.0   9.0   3.0  6.0  12.0   5.9   
 2    4         Spain  2021  6.0  2.0  4.0  0.0  13.0   6.0  7.0  10.0  16.7   
 3    5       Denmark  2021  6.0  3.0  0.0  3.0  12.0   7.0  5.0   9.0   9.9   
 8    6   Netherlands  2021  4.0  3.0  0.0  1.0   8.0   4.0  4.0   9.0   8.6   
 5    7       Czechia  2021  5.0  2.0  1.0  2.0   6.0   4.0  2.0   7.0   6.2   
 9    8        Sweden  2021  4.0  2.0  1.0  1.0   5.0   4.0  1.0   7.0   5.2   
 10   9        France  2021  4.0  1.0  3.0  0.0   7.0   6.0  1.0   6.0   7.5   
 11  10       Austria  2021  4.0  2.0  0.0  2.0   5.0   5.0  0.0   6.0   5.1   
 6   11   Switzerland  2021  5.0  1.0  3.0  1.0   8.0   9.0 -1.0   6.0   7.1   
 7   12       Ukraine  2021  5.0  2.0  0

In [31]:
all_matches_df = pd.concat(all_matches)

In [32]:
all_matches_df.columns = [c.lower() for c in all_matches_df.columns]

In [33]:
columns_to_drop = ['notes', 'xg', 'xga', 'xga', 'xgd', 'xgd/90']
new_all_matches_df = all_matches_df.drop(columns_to_drop, axis='columns')

In [55]:
new_all_matches_df.to_csv("matches.csv")

In [56]:
# Nun neben wir von WM-2022 die Einzelspieler Stats
all_players = []

links = ['World Cup Player Stats.html', 'UEFA Euro Qualifying Player Stats.html',]

for link in links:
    with open(link, 'r', encoding='utf-8') as file:
            html_content = file.read()

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'lxml')

    player_table = soup.select("table.stats_table")[0]

    player_table

    player_df = pd.read_html(StringIO(str(player_table)))[0]

    player_df.columns = player_df.columns.droplevel()

    player_df['Squad'] = player_df['Squad'].apply(lambda x: x[3:].strip() if isinstance(x, str) and len(x) > 3 else x)

    all_players.append(player_df)

    player_df.head()

In [62]:
all_players[0]

,Rk,Player,Pos,Squad,Age,Club,Born,MP,Starts,Min,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,1,Brenden Aaronson,MF,United States,21,1.eng Leeds United,2000,4,0,108,...,0.00,0.00,0.00,0.00,0.06,0.18,0.24,0.06,0.24,Matches
1,2,Ali Abdi,DF,Tunisia,28,2.fr Caen,1993,3,2,188,...,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.02,Matches
2,3,Salis Abdul Samed,MF,Ghana,22,1.fr Lens,2000,3,3,250,...,0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.01,0.02,Matches
3,4,Saud Abdulhamid,"DF,MF",Saudi Arabia,22,1.sa Al-Hilal,1999,3,3,270,...,0.00,0.00,0.00,0.00,0.01,0.02,0.02,0.01,0.02,Matches
4,5,Nawaf Al Abed,MF,Saudi Arabia,32,1.sa Al-Shabab,1990,2,0,88,...,0.00,0.00,0.00,0.00,0.13,0.06,0.20,0.13,0.20,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,676,Piotr Zieliński,"MF,FW",Poland,28,1.it Napoli,1994,4,4,328,...,0.00,0.27,0.27,0.27,0.16,0.11,0.26,0.16,0.26,Matches
676,677,Walker Zimmerman,DF,United States,29,1.us Nashville,1993,4,3,279,...,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.02,0.02,Matches
677,678,Andrija Živković,DF,Serbia,25,1.gr PAOK,1996,3,3,210,...,0.86,0.86,0.00,0.86,0.03,0.48,0.50,0.03,0.50,Matches
678,679,Hakim Ziyech,FW,Morocco,29,1.eng Chelsea,1993,7,7,635,...,0.14,0.28,0.14,0.28,0.06,0.10,0.16,0.06,0.16,Matches


In [63]:
all_players[1]

,Rk,Player,Pos,Squad,Age,Born,MP,Starts,Min,90s,...,PK,PKatt,CrdY,CrdR,Gls,Ast,G+A,G-PK,G+A-PK,Matches
0,1,Bárður Á Reynatrøð,GK,Faroe Islands,24,2000,1,0,57,0.6,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,Matches
1,2,Liel Abada,FW,Israel,22,2001,2,1,65,0.7,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,Matches
2,3,Mohammad Abu Fani,MF,Israel,25,1998,8,2,318,3.5,...,0,0,1,0,0.00,0.00,0.00,0.00,0.00,Matches
3,4,Giorgi Aburjania,MF,Georgia,29,1995,2,2,134,1.5,...,0,0,2,0,0.00,0.00,0.00,0.00,0.00,Matches
4,5,Francesco Acerbi,DF,Italy,35,1988,4,4,332,3.7,...,0,0,2,0,0.00,0.00,0.00,0.00,0.00,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,1629,Viktor Đukanović,FW,Montenegro,20,2004,2,0,30,0.3,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,Matches
1694,1630,Filip Đukić,GK,Montenegro,24,1999,1,0,33,0.4,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,Matches
1695,1631,Filip Đuričić,MF,Serbia,32,1992,3,2,130,1.4,...,0,0,0,0,0.00,0.00,0.00,0.00,0.00,Matches
1696,1632,Uroš Đurđević,FW,Montenegro,29,1994,2,1,64,0.7,...,0,0,2,0,0.00,0.00,0.00,0.00,0.00,Matches


In [78]:
# Annahme: all_players ist eine Liste mit zwei DataFrames
df1 = all_players[0]
df2 = all_players[1]

# Gemeinsame Spalten finden
common_columns = list(set(df1.columns) & set(df2.columns))

# DataFrames auf gemeinsame Spalten reduzieren
df1 = df1[common_columns]
df2 = df2[common_columns]

# DataFrames kombinieren
combined_df = pd.concat([df1, df2])

In [82]:
# Bei doppelte vorkommende Spieler werden nur der besten Stats-Jahr genommen

import numpy as np

# Funktion zur Auswahl der besseren Statistiken
def select_better_stats(group):
    result = {}
    for column in group.columns:
        if pd.api.types.is_numeric_dtype(group[column]):
            # Für numerische Spalten nehmen wir den Maximalwert
            result[column] = group[column].max()
        else:
            # Für nicht-numerische Spalten nehmen wir den ersten nicht-NaN Wert
            result[column] = group[column].dropna().iloc[0] if not group[column].dropna().empty else np.nan
    return pd.Series(result)

combined_df = combined_df.groupby('Player', as_index=False).apply(select_better_stats)

combined_df.shape

(2098, 24)

In [83]:
# Spalten mit allen NaN-Werten entfernen
combined_df = combined_df.dropna(axis=1, how='all')

combined_df.shape

(2098, 24)

In [92]:
combined_df.head()

,Min,Matches,Squad,G+A_1,Rk,CrdR,Starts,PK,G-PK,90s,...,CrdY,MP,G+A-PK,Pos,Gls_1,Age,Gls,Player,G+A,Ast_1
0,13,Matches,Rep. of Ireland,0.00,288,0,0,0,0,0.1,...,0,1,0.00,FW,0.00,24,0,Aaron Connolly,0,0.00
1,513,Matches,Scotland,0.00,564,0,6,0,0,5.7,...,1,6,0.00,"MF,DF",0.00,21,0,Aaron Hickey,0,0.00
2,360,Matches,Australia,0.0,424,0,4,0,0,4.0,...,1,4,0.0,MF,0.0,31,0,Aaron Mooy,0,0.0
3,266,Matches,Wales,0.0,504,0,3,0,0,3.0,...,1,3,0.0,MF,0.0,31,0,Aaron Ramsey,0,0.0
4,139,Matches,Andorra,0.00,1255,0,1,0,0,1.5,...,2,4,0.00,FW,0.00,27,0,Aaron Sánchez,0,0.00


In [93]:
combined_df.columns = [c.lower() for c in combined_df.columns]

In [95]:
combined_df.columns 

Index(['min', 'matches', 'squad', 'g+a_1', 'rk', 'crdr', 'starts', 'pk',
       'g-pk', '90s', 'ast', 'born', 'g-pk_1', 'pkatt', 'crdy', 'mp', 'g+a-pk',
       'pos', 'gls_1', 'age', 'gls', 'player', 'g+a', 'ast_1'],
      dtype='object')

In [103]:
columns_to_drop = ['matches', 'rk', 'born', '90s', 'pkatt', 'pos', 'age']
new_combined_df = combined_df.drop(columns_to_drop, axis='columns')

In [104]:
new_combined_df.columns 

Index(['min', 'squad', 'g+a_1', 'crdr', 'starts', 'pk', 'g-pk', 'ast',
       'g-pk_1', 'crdy', 'mp', 'g+a-pk', 'gls_1', 'gls', 'player', 'g+a',
       'ast_1'],
      dtype='object')

In [105]:
new_combined_df.shape

(2098, 17)

In [106]:
new_combined_df.to_csv("players.csv")

In [107]:
new_combined_df.head()

,min,squad,g+a_1,crdr,starts,pk,g-pk,ast,g-pk_1,crdy,mp,g+a-pk,gls_1,gls,player,g+a,ast_1
0,13,Rep. of Ireland,0.00,0,0,0,0,0,0.00,0,1,0.00,0.00,0,Aaron Connolly,0,0.00
1,513,Scotland,0.00,0,6,0,0,0,0.00,1,6,0.00,0.00,0,Aaron Hickey,0,0.00
2,360,Australia,0.0,0,4,0,0,0,0.0,1,4,0.0,0.0,0,Aaron Mooy,0,0.0
3,266,Wales,0.0,0,3,0,0,0,0.0,1,3,0.0,0.0,0,Aaron Ramsey,0,0.0
4,139,Andorra,0.00,0,1,0,0,0,0.00,2,4,0.00,0.00,0,Aaron Sánchez,0,0.00
